In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats

import seaborn as sns


In [ ]:


### execute script to load modules here
exec(open('setup_aesthetics.py').read())

## update locally to have larger axes titles
rcParams['axes.titlesize'] = MEDIUM_SIZE + 2
sns.set_theme(style = 'ticks',font_scale = 1.5,rc=rcParams)

In [ ]:
## create export directory if necessary
## foldernames for output plots/lists produced in this notebook
import os
FIG_DIR = f'./figures/decorrelation_isoclines/'
os.makedirs(FIG_DIR, exist_ok=True)
print("All  plots will be stored in: \n" + FIG_DIR)

## plot isoclines for selection coefficient

In [ ]:
def logfoldchange2logit_coefficient(fc_wt, fc_mut):
    return fc_mut - fc_wt
    

def logfoldchange2ltee_coefficient(fc_wt,fc_mut):
    return fc_mut/fc_wt

In [ ]:
def get_isocline_logit(fc_wt, level):
    return fc_wt + level

### test
get_isocline_logit(fc_wt = np.geomspace(1.1,100,num=10), level = 1)

In [ ]:
def get_isocline_ltee(fc_wt, level):
    return fc_wt*level

### test
get_isocline_ltee(fc_wt = np.geomspace(1.1,100,num=10), level = 2)

### Sample a bivariate set of frequencies

In [ ]:

np.random.seed(29071997)

In [ ]:
# choose a schwerpunk in the space
mean = (6.7,7.2) #x0,xf

# sample a set of wildtype and mutant foldchange
fcwt_sample, fcwt_sample= np.random.multivariate_normal(mean, cov=0.01*np.eye(2),size=100).T


In [ ]:
np.log(1e4)

In [ ]:
angles = np.linspace(np.pi/3,np.pi/2 - 0.001, num = 6)

In [ ]:
angles

In [ ]:
np.tan(angles)

In [ ]:
levels = np.outer([-1,1],np.tan(angles))

In [ ]:
levels.flatten()

In [ ]:
fig, ax = plt.subplots(figsize = (FIGHEIGHT_TRIPLET,FIGHEIGHT_TRIPLET))


fcmax = 6
fcwt_vec = np.linspace(0.01,fcmax, num = 100) 
fcwt_vec = np.concatenate((-fcwt_vec,fcwt_vec))
color_logit = 'tab:grey'
color_ltee = 'navy'

### plot logit isoclines
levels = np.outer([-1,1],np.linspace(0.01,10,num = 6)).flatten()
for level in levels: 

    y = get_isocline_logit(fc_wt = fcwt_vec, level = level)
    ax.plot(fcwt_vec, y, color = color_logit)
    
    
## plot ltee isoclines
angles = np.linspace(np.pi/2,np.pi - 0.001, num = 6)
levels = np.outer([-1,1],np.tan(angles)).flatten()

for level in levels: 
    y = get_isocline_ltee(fc_wt = fcwt_vec, level = level) # need to account for different neutral point in W
    ax.plot(fcwt_vec, y, color = color_ltee)
    
## plot diagonal 
ax.plot([-fcmax,fcmax],[-fcmax,fcmax], color = 'red', ls = '--', label = 'y=x')

## add  legend items
ax.plot([],[], color = color_logit, label = 'logit $s$ isocline')
ax.plot([],[], color = color_ltee, label = 'LTEE $W$ isocline')

### plot cloud of points

#ax.scatter(x0_sample,xf_sample, color = 'tab:orange', marker = 'o', s = 5)

ax.set_xlim(-fcmax,fcmax)
ax.set_ylim(-fcmax,fcmax)
ax.legend(loc = 'upper left', bbox_to_anchor = (1.1,1.))

ax.set_xlabel('wild-type fold-change\n$\log\; N_{wt}(t_f)/N_{wt}(0)$')
ax.set_ylabel('mutant fold-change\n$\log\;  N_{mut}(t_f)/N_{mut}(0)$')

fig.savefig(FIG_DIR + f'isoclines_logit_vs_ltee.pdf', DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)
              

### plot conditions for misranking

In [ ]:
fig, ax = plt.subplots(figsize = (FIGHEIGHT_TRIPLET,FIGHEIGHT_TRIPLET))

### plot diagonal
x_vec = np.linspace(0,np.log(200), num = 50)
#ax.plot(x_vec,x_vec, ls = '-', color = 'tab:grey')

ax.set_xlim(x_vec.min(),x_vec.max())
ax.set_ylim(x_vec.min(),1.8*x_vec.max())

## set labels
ax.set_xlabel(r"wild-type log fold-change: $\mathrm{LFC}_{\mathrm{wt}}$")
ax.set_ylabel(r"mutant log fold-change: $\mathrm{LFC}_{\mathrm{mut}}$")

## take off axis spines
#sns.despine(left=False, bottom = False, ax = ax)

### plot example point
A, B = 1,2.5
ax.scatter([A], [B], color = 'tab:red', s = 70, zorder = 3)
s = B-A
q = s/A

### plot area of decorrelation
ax.plot(x_vec, B/A*x_vec, color = 'tab:blue')
ax.plot(x_vec, (x_vec-A) + B, color = 'tab:grey', ls = '--')

### plot area
x_fill = np.linspace(x_vec.min(),x_vec.max())
y_fill = B/A*x_fill
ax.fill_between(x_fill, (x_fill - A) + B, y_fill, color='tab:red', alpha=0.2)

### plot isoclines for W
#ax.plot(x_vec, (x_vec-A) + 1.7*B, color = 'tab:grey', ls = '--')
#ax.plot(x_vec, (x_vec-A) + 2.4*B, color = 'tab:grey', ls = '--')

## plot isoclines for s
#ax.plot(x_vec, 0.7*B/A*x_vec, color = 'tab:blue', ls = '-')

### plot intersection
A_misranked = (A- 1.7*B)/(1-0.7*B/A)
B_misranked = (A_misranked - A) + 1.7*B
ax.scatter(A_misranked, B_misranked , color = 'tab:grey', s = 70, zorder = 3)
s_misranked = B_misranked - A_misranked
q_misranked = s_misranked/A_misranked


### plot inset
inset = ax.inset_axes(bounds = [0.7,0.15,0.25,0.25]) # lower right
#inset = ax.inset_axes(bounds = [0.15,0.75,0.25,0.25]) # upper left
inset.set_xlabel(r'$s^{\mathrm{logit}}_{\mathrm{cycle}}$')
inset.set_ylabel(''+r'$s^{\mathrm{logit}}_{\mathrm{gen}}$')
inset.tick_params(bottom = False,labelbottom =False, left = False, labelleft = False)

inset.scatter([s],[q], color = 'tab:red')
inset.scatter([s_misranked],[q_misranked], color = 'tab:grey')
inset.set_xlim(s-0.4,s_misranked+0.4)
inset.set_ylim(q_misranked*0.8, q*1.2,)

fig.savefig(FIG_DIR + f'scatterplot_misranking_condition.pdf', DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)


### plot variation with small mutant frequency x

In [ ]:
## create variation in 
B_vec = np.linspace(B,8, num = 10)
A_vec = A*np.ones_like(B_vec)

s_vec = B_vec - A_vec
q_vec = np.divide(s_vec,A_vec)

In [ ]:
fig, ax = plt.subplots(figsize = (FIGHEIGHT_TRIPLET,FIGHEIGHT_TRIPLET))

### plot diagonal
ax.plot(x_vec,x_vec, ls = '-', color = 'tab:grey')

ax.set_xlim(x_vec.min(),x_vec.max())
ax.set_ylim(x_vec.min(),1.8*x_vec.max())

## set labels
ax.set_xlabel(r"wild-type fold-change $\log\; A$")
ax.set_ylabel(r"mutant fold-change $\log\; B$")

## take off axis spines
#sns.despine(left=False, bottom = False, ax = ax)

### plot distribution of points

ax.scatter(A_vec, B_vec, color = 'tab:grey', s = 70)

### plot area of decorrelation
for i in [2,6]:
    A, B = A_vec[i], B_vec[i]
    x_fill = np.linspace(A,x_vec[-11])
    y_fill = B/A*x_fill
    ax.fill_between(x_fill, (x_fill - A) + B, y_fill, color='tab:red', alpha=0.2)



### plot inset
inset = ax.inset_axes(bounds = [0.7,0.1,0.25,0.25])
sns.despine(left= False, bottom = False, ax =inset)
inset.set_xlabel('fitness $s$')
inset.set_ylabel('fitness $q$')
inset.tick_params(bottom = False,labelbottom =False, left = False, labelleft = False)

inset.scatter(s_vec,q_vec, color = 'tab:grey')


fig.savefig(FIG_DIR + f'scatterplot_variation_in_wild-type_foldchange_only.pdf', DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)


### plot variation with fold-change condition

In [ ]:
def wt2mut_foldchange(A, x = 0.5, Y = (1, 0.5), R_0=1,N_0 = 0.01):
    Y_bar = 1/((1-x)/Y[0] + x/Y[1])
    C = (R_0*Y_bar)/N_0  + 1
    
    B = np.log(Y[0]/x/Y_bar) + np.log(C - ((1-x)*Y_bar/Y[1])*np.exp(A))
    
    return B

### test

wt2mut_foldchange(A = np.log(100),x = 0.5, Y = (1.,1.), R_0 = 1, N_0 = 0.01)

In [ ]:
## sample with more points
#params = {'x' : 0.5, 'Y' : (1.,0.5), 'R_0' : 1, 'N_0' : 0.1 }
#A_vec = np.linspace(0.25,5, num = 30)
#B_vec = np.array([wt2mut_foldchange(A, **params) for A in A_vec])

## sample with fewer points
params = {'x' : 0.5, 'Y' : (1.,0.5), 'R_0' : 1, 'N_0' : 0.1 }
A_vec = np.linspace(0.5,4, num = 10)
B_vec = np.array([wt2mut_foldchange(A, **params) for A in A_vec])



s_vec = B_vec - A_vec
q_vec = np.divide(s_vec,A_vec)

In [ ]:
fig, ax = plt.subplots(figsize = (FIGHEIGHT_TRIPLET,FIGHEIGHT_TRIPLET))

### plot diagonal
#ax.plot(x_vec,x_vec, ls = '-', color = 'tab:grey')

ax.set_xlim(x_vec.min(),x_vec.max())
ax.set_ylim(x_vec.min(),1.8*x_vec.max())

## set labels
ax.set_xlabel(r"wild-type log fold-change: $\mathrm{LFC}_{\mathrm{wt}}$")
ax.set_ylabel(r"mutant log fold-change: $\mathrm{LFC}_{\mathrm{mut}}$")

## take off axis spines
#sns.despine(left=False, bottom = False, ax = ax)

### plot distribution of points
ax.scatter(A_vec, B_vec, color = 'tab:grey', s = 70,zorder = 3)

### plot area of decorrelation
select = [1,2]
colors = ['tab:red', 'tab:red']
for i,c in zip(select, colors):
    A, B = A_vec[i], B_vec[i]
    ax.scatter(A, B, color = c, s = 70,zorder = 3)
    x_fill = np.linspace(x_vec[0],x_vec[-1])
    y_fill = B/A*x_fill
    ax.fill_between(x_fill, (x_fill - A) + B, y_fill, color='tab:red', alpha=0.2)

## plot line
ax.plot(x_vec,[ wt2mut_foldchange(A,**params) for A in x_vec], color = 'black')

### plot inset
inset = ax.inset_axes(bounds = [0.7,0.15,0.25,0.25]) # lower right
#inset = ax.inset_axes(bounds = [0.15,0.75,0.25,0.25]) # upper left
inset.set_xlabel(r'$s^{\mathrm{logit}}_{\mathrm{cycle}}$')
inset.set_ylabel(''+r'$s^{\mathrm{logit}}_{\mathrm{gen}}$')
inset.tick_params(bottom = False,labelbottom =False, left = False, labelleft = False)


inset.scatter(s_vec,q_vec, color = 'tab:grey')
inset.scatter(s_vec[select],q_vec[select], color = 'tab:red')

fig.savefig(FIG_DIR + f'scatterplot_variation_mass_balance_only.pdf', DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)


### Create variation in the biomass yield

### random sample

mean = [1.5,3]
cov = [[0.4/1.8,0.0],[0.0,0.4]]

np.random.seed([6754243])

sample = np.random.multivariate_normal(mean, cov, 20)
A_vec,B_vec = sample.T

### systematic samplee

def circle_points(r, n):
    t = np.linspace(0, 2*np.pi, n, endpoint=False)
    x = r * np.cos(t)
    y = r/2. * np.sin(t)
    return x,y

circle = circle_points(1,8)

angle = 0
rotation = np.array([[np.cos(angle),np.sin(angle)], [np.sin(angle),np.cos(angle)]])

dots = np.dot(rotation, circle)

A_vec = dots[0]+1.5
B_vec = dots[1]+3 

In [ ]:
### systematic sample

A_vec = np.array([])
B_vec = np.array([])

for x,offset in zip([np.linspace(0.8,1.5,3),np.linspace(1.,1.7,3),np.linspace(1.2,1.9,3)],
                    [0,0.6,1.2]):
    y = -0.25*x+3 + offset
    B_vec = np.hstack([B_vec,y])
    A_vec = np.hstack([A_vec,x])

In [ ]:

s_vec = B_vec - A_vec
q_vec = np.divide(s_vec,A_vec)

In [ ]:
fig, ax = plt.subplots(figsize = (FIGHEIGHT_TRIPLET,FIGHEIGHT_TRIPLET))

### plot diagonal
#ax.plot(x_vec,x_vec, ls = '-', color = 'tab:grey')

ax.set_xlim(x_vec.min(),x_vec.max())
ax.set_ylim(x_vec.min(),1.8*x_vec.max())

## set labels
ax.set_xlabel(r"wild-type log fold-change: $\mathrm{LFC}_{\mathrm{wt}}$")
ax.set_ylabel(r"mutant log fold-change: $\mathrm{LFC}_{\mathrm{mut}}$")

## take off axis spines
#sns.despine(left=False, bottom = False, ax = ax)

### plot distribution of points
ax.scatter(A_vec, B_vec, color = 'tab:grey', s = 70,zorder = 3)

### plot area of decorrelation
select = [0]
for i in select:
    A, B = A_vec[i], B_vec[i]
    ax.scatter(A, B, color = 'tab:red', s = 70,zorder = 3)
    x_fill = np.linspace(x_vec[0],x_vec[-1])
    y_fill = B/A*x_fill
    ax.fill_between(x_fill, (x_fill - A) + B, y_fill, color='tab:red', alpha=0.2)

## plot line
#ax.plot(x_vec,[ wt2mut_foldchange(A,**params) for A in x_vec], color = 'black')

### plot inset
inset = ax.inset_axes(bounds = [0.7,0.15,0.25,0.25]) # lower right
#inset = ax.inset_axes(bounds = [0.15,0.75,0.25,0.25]) # upper left
inset.set_xlabel(r'$s^{\mathrm{logit}}_{\mathrm{cycle}}$')
inset.set_ylabel(''+r'$s^{\mathrm{logit}}_{\mathrm{gen}}$')
inset.tick_params(bottom = False,labelbottom =False, left = False, labelleft = False)


inset.scatter(s_vec,q_vec, color = 'tab:grey')
inset.scatter(s_vec[select],q_vec[select], color = 'tab:red')
fig.savefig(FIG_DIR + f'scatterplot_variation_no_constraint.pdf', DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)


## create variation around a fixed budget

In [ ]:
### systematic sample

A_vec = np.array([])
B_vec = np.array([])

#for x,offset in zip([np.linspace(0.5,1.5,5),np.linspace(0.7,1.7,5),np.linspace(0.9,1.9,5)], [0,0.6,1.2]):
    
for x,offset in zip([np.linspace(0.6,1.6,4),np.linspace(1.,2.,4)], [0,1.2]):
    y = -0.5*x+2.7 + offset
    B_vec = np.hstack([B_vec,y])
    A_vec = np.hstack([A_vec,x])

In [ ]:
## calculate fold-change, using mass conservation
params = {'x' : 0.5, 'Y' : (1.,0.5), 'R_0' : 1, 'N_0' : 0.1 }
A_vec_new = np.linspace(0.5,4, num = 10)
A_vec = np.hstack([A_vec,A_vec_new])
B_vec = np.hstack([B_vec,np.array([wt2mut_foldchange(A, **params) for A in A_vec_new])])


s_vec = B_vec - A_vec
q_vec = np.divide(s_vec,A_vec)

In [ ]:
## set color for the points

In [ ]:
fig, ax = plt.subplots(figsize = (FIGHEIGHT_TRIPLET,FIGHEIGHT_TRIPLET))

### plot diagonal
#ax.plot(x_vec,x_vec, ls = '-', color = 'tab:grey')

ax.set_xlim(x_vec.min(),x_vec.max())
ax.set_ylim(x_vec.min(),1.8*x_vec.max())

## set labels
ax.set_xlabel(r"wild-type log fold-change: $\mathrm{LFC}_{\mathrm{wt}}$")
ax.set_ylabel(r"mutant log fold-change: $\mathrm{LFC}_{\mathrm{mut}}$")

## take off axis spines
#sns.despine(left=False, bottom = False, ax = ax)

### plot distribution of points
ax.scatter(A_vec, B_vec, color = 'tab:grey', s = 70,zorder = 3)

### plot area of decorrelation
select = [0]
for i in select:
    A, B = A_vec[i], B_vec[i]
    ax.scatter(A,B,s=70,color ='tab:red', zorder = 3)
    x_fill = np.linspace(x_vec[0],x_vec[-1])
    y_fill = B/A*x_fill
    ax.fill_between(x_fill, (x_fill - A) + B, y_fill, color='tab:red', alpha=0.2)
    

## plot line
ax.plot(x_vec,[ wt2mut_foldchange(A,**params) for A in x_vec], color = 'black')

### plot inset
inset = ax.inset_axes(bounds = [0.7,0.15,0.25,0.25]) # lower right
#inset = ax.inset_axes(bounds = [0.15,0.75,0.25,0.25]) # upper left
inset.set_xlabel(r'$s^{\mathrm{logit}}_{\mathrm{cycle}}$')
inset.set_ylabel(''+r'$s^{\mathrm{logit}}_{\mathrm{gen}}$')
inset.tick_params(bottom = False,labelbottom =False, left = False, labelleft = False)


inset.scatter(s_vec,q_vec, color = 'tab:grey')
inset.scatter(s_vec[select],q_vec[select], color ='tab:red', zorder = 3)


### plot the points inside the cone
assert len(select) ==1 
## find points in the cone
list_inside = []

for i in range(len(A_vec)):
    A, B = A_vec[i], B_vec[i]
    delta_s = s_vec[i] - s_vec[select]
    delta_q = q_vec[i] - q_vec[select]
    if delta_s > 0 and delta_q < 0:
        list_inside.append(i)

## choose colors for the points int he cone
list_colors = ['tab:blue', 'seagreen', 'darkorange']

for i, c in zip(list_inside, list_colors):
    ax.scatter(A_vec[i], B_vec[i], s=70, zorder = 5, color = c)
    inset.scatter(s_vec[i], q_vec[i], zorder =5, color = c)

fig.savefig(FIG_DIR + f'scatterplot_variation_around_budget.pdf', DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)


### plot variation with fold-change condition and variation in Yield

In [ ]:
fig, ax = plt.subplots(figsize = (FIGHEIGHT_TRIPLET,FIGHEIGHT_TRIPLET))

### plot diagonal
ax.plot(x_vec,x_vec, ls = '-', color = 'tab:grey')

ax.set_xlim(x_vec.min(),x_vec.max())
ax.set_ylim(x_vec.min(),1.8*x_vec.max())

## set labels
ax.set_xlabel(r"wild-type foldchange $\log\; \mathrm{A}$")
ax.set_ylabel(r"mutant foldchange $\log\; \mathrm{B}$")

## take off axis spines
#sns.despine(left=False, bottom = False, ax = ax)


### plot inset
#inset = ax.inset_axes(bounds = [0.75,0.1,0.25,0.25])
#inset = ax.inset_axes(bounds = [0.15,0.75,0.25,0.25])
inset = ax.inset_axes(bounds = [1.15,0.75,0.25,0.25])
sns.despine(left= False, bottom = False, ax =inset)
inset.set_xlabel('fitness $s$')
inset.set_ylabel('fitness $q$')
inset.tick_params(bottom = False,labelbottom =False, left = False, labelleft = False)

    
for R_zero, color, yinset  in zip([2,1.], ['blue', 'darkorange'],[0.1,0.4]): #[1.,5.,50.]:
    params['R_0'] = R_zero
    B_vec = np.array([wt2mut_foldchange(A, **params) for A in A_vec])
   
    s_vec = B_vec - A_vec
    q_vec = np.divide(s_vec,A_vec)

    ### plot distribution of points
    ax.scatter(A_vec, B_vec, color = color, s = 70)

    ### plot area of decorrelation
    for i in [3]:
        A, B = A_vec[i], B_vec[i]
        x_fill = np.linspace(A,x_vec[-1])
        y_fill = B/A*x_fill
        ax.fill_between(x_fill, (x_fill - A) + B, y_fill, color='tab:red', alpha=0.2)

    inset.scatter(s_vec,q_vec, color = color)
    
    ## plot correlation within
    within = ax.inset_axes(bounds = [1.15,yinset,0.25,0.25])
    within.scatter(s_vec,q_vec, color = color)
    within.set_xlabel('fitness $s$')
    within.set_ylabel('fitness $q$')
    within.tick_params(bottom = False,labelbottom =False, left = False, labelleft = False)
    
    ## plot line
    ax.plot(x_vec,[ wt2mut_foldchange(A,**params) for A in x_vec], color = 'black')


fig.savefig(FIG_DIR + f'scatterplot_variation_mass_balance_with_yield_variation.pdf', DPI = DPI, bbox_inches = 'tight', pad_inches = PAD_INCHES)
